In [1]:
#ignore warnings for clear printing
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns # More snazzy plotting library
import itertools

file_path = "/home/user/projects/data_mining/predictive_data_mining/logistic_regression/working_dataset.csv"
dta = pd.read_csv(file_path)
#clean up data non numeric rows
str_list = [] # empty list to contain columns with strings (words)
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str:
        #if colname not in str_list:
            str_list.append(colname)
# Get to the numeric columns by inversion
num_list = dta.columns.difference(str_list)
#USe only the numeriv values
dta_clean = dta[num_list]
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
#dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.reindex_axis(sorted(dta_clean.columns), axis=1)
#clean up data from zero rows 
for colname, colvalue in dta_clean.iteritems():
    if colname != 'facenumber_in_poster':
        dta_clean = dta_clean[dta_clean[colname] != 0]
dta_clean.count()
#clasify the data for the logistic regression
def label_gross (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 10000000)) : return 2
    elif ((gross >= 10000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 200000000)) : return 4
    elif (gross >= 200000000) : return 5

y_target = dta_clean.gross.apply (lambda gross: label_gross (gross))
y_target = np.ravel(y_target)
x_data = dta_clean.drop('gross', axis=1)
#x_data = x_data.drop('aspect_ratio', axis=1)
#x_data = x_data.drop('duration', axis=1)
#x_data = x_data.drop('director_facebook_likes', axis=1)
dta_clean['gross_class'] = dta_clean.gross.apply (lambda gross: label_gross (gross))

#standarlisze
for colname, colvalue in x_data.iteritems():
                standard_scaler = preprocessing.StandardScaler().fit(x_data[colname])
                x_data[colname] = standard_scaler.transform(x_data[colname])


#calculate number of different componets in polynomal transfor
#poly_n_output_features = []
#for deg in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
#    poly = PolynomialFeatures(deg, interaction_only=True).fit(x_data)
#    poly_n_output_features.append(poly.n_output_features_)
#print poly_n_output_features
poly_n_output_features = [16, 136, 816, 3876, 15504, 54264, 170544, 490314, 1307504, 3268760, 7726160, 17383860, 37442160, 77558760, 155117520, 300540195]
poly_n_output_features_inter_only = [16, 121, 576, 1941, 4944, 9949, 16384, 22819, 27824, 30827, 32192, 32647, 32752, 32767, 32768, 32768]
def run_grid_search_pca(x,y,name, degrees, n_components, solvers, class_weight, Cs):
    #create pipline and use GridSearch to find the betst params
    polynomial = PolynomialFeatures().fit(x_data)
    pca = decomposition.PCA().fit(x_data) 
    logistic = linear_model.LogisticRegression()
    pipe = Pipeline(steps=[('polynomial', polynomial),('pca', pca), ('logistic', logistic)])
    #create estimator
    estimator = GridSearchCV(pipe,
                             dict(polynomial__degree = degrees,
                                  pca__n_components = n_components,
                                  logistic__C = Cs,                           
                                  logistic__solver = solvers,
                                  logistic__class_weight = class_weight)
                             ,verbose=2, cv=5, n_jobs=4)
    #run the esmimator    
    estimator.fit(x, y)
    print "GREP_ME***Results of ["  + name + "] estimator run are"
    print estimator.cv_results_
    print "GREP_ME***Best params of ["  + name + "] estimator run are"
    print estimator.best_params_
    print "GREP_ME***Best score of ["  + name + "] estimator run are"
    print estimator.best_score_
    
def run_grid_search_fa(x,y,name, degrees, n_components, solvers, class_weight, Cs):
    #create pipline and use GridSearch to find the betst params
    polynomial = PolynomialFeatures().fit(x_data)
    fa = decomposition.FactorAnalysis().fit(x_data) 
    logistic = linear_model.LogisticRegression()
    pipe = Pipeline(steps=[('polynomial', polynomial),('fa', fa), ('logistic', logistic)])
    #create estimator
    estimator = GridSearchCV(pipe,
                             dict(polynomial__degree = degrees,
                                  fa__n_components = n_components,
                                  logistic__C = Cs,                           
                                  logistic__solver = solvers,
                                  logistic__class_weight = class_weight)
                             ,verbose=2, cv=5, n_jobs=4)
    #run the esmimator    
    estimator.fit(x, y)
    print "GREP_ME***Results of ["  + name + "] estimator run are"
    print estimator.cv_results_
    print "GREP_ME***Best params of ["  + name + "] estimator run are"
    print estimator.best_params_
    print "GREP_ME***Best score of ["  + name + "] estimator run are"
    print estimator.best_score_

def run():
    ###################################################
    # Process a simple first degree pipeline with PCA #
    ###################################################
    degrees=[1]
    n_components = [4, 5, 6, 7, 8, 9, 10, 11, 12, 15]
    solvers =  ['newton-cg', 'lbfgs', 'liblinear', 'sag']
    class_weight = [None, "balanced"]
    Cs = np.logspace(-4, 4, 3)
    run_grid_search_pca(x_data, y_target, "First degree, PCA - no mle", degrees, n_components, solvers, class_weight, Cs)

    ###############################################
    # Process a 5 degree pipeline with  PCA mle  #
    ###############################################
    degrees=[1,2,3,4,5]
    n_components = ['mle']
    solvers =  ['newton-cg', 'lbfgs', 'liblinear', 'sag']
    class_weight = [None, "balanced"]
    Cs = np.logspace(-4, 4, 3)
    run_grid_search_pca(x_data, y_target, "5 degrees, PCA with mle", degrees, n_components, solvers, class_weight, Cs)

    ##########################################
    # Process a 5 degree pipeline with FA   #
    ##########################################
    for deg in [1, 2,3,4,5]:
        degrees=[deg]
        n_components = [4, 5, 6, 7, 8, 9, 10, 11, 12, poly_n_output_features[deg-1] - 1]
        solvers =  ['newton-cg', 'lbfgs', 'liblinear', 'sag']
        class_weight = [None, "balanced"]
        Cs = np.logspace(-4, 4, 3)
        print n_components
        string = str(deg) + " degree, FA"
        run_grid_search_fa(x_data, y_target, string, degrees, n_components, solvers, class_weight, Cs)

#ignore warnigs
#with warnings.catch_warnings():
    #warnings.simplefilter("ignore")
    #run()


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either usi

plt.plot(np.cumsum(pca.explained_variance_ratio_), linewidth=2)
plt.xlabel('n_components')
plt.axis('tight')
plt.ylabel('explained_variance_')


In [ ]:
plt.axvline(estimator.best_estimator_.named_steps['pca'].n_components, linestyle=':', label='n_components chosen')
plt.legend(prop=dict(size=12))
plt.show()

In [ ]:
estimator.best_params_

In [ ]:
estimator.best_estimator_

In [ ]:
#

In [6]:
pcaa = decomposition.PCA(n_components=15)
x_9d = pcaa.fit_transform(x_data)
pd.DataFrame(x_9d)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,16.879704,0.774671,3.254583,2.058925,-1.083469,-4.067253,-3.712747,-1.810506,0.166111,0.835892,0.443612,-3.196898,0.198663,0.431572,1.018338
1,1.439405,-1.768893,-1.395954,-0.002661,0.696795,-0.031258,-0.368258,-0.422024,-0.541979,0.354348,-0.543080,0.205380,-1.074646,-0.041857,-0.051882
2,0.587581,-1.556440,-0.417946,0.922851,0.224189,-0.789379,1.373897,-0.309675,0.079547,-0.514781,-0.365063,-0.054898,-0.053710,-0.519767,-0.005765
3,5.341091,2.447035,1.136194,-0.572845,1.136502,-0.290321,-0.881474,-1.625177,-0.766575,-0.464617,-1.415081,-1.164149,-1.026877,0.075728,0.335279
4,4.888255,-0.187847,1.105948,-0.433291,0.695658,1.522818,0.172693,1.242736,-2.792377,0.030659,-2.257256,0.406169,-0.175680,0.808064,0.008926
5,3.675436,0.629586,-0.866227,-0.057936,0.615655,2.073708,0.782912,0.497105,-0.714937,1.483942,-0.155401,-0.415624,-0.712512,0.256229,0.047056
6,4.684932,1.817417,-0.604339,-0.560701,0.948505,0.795134,1.252543,0.568650,-0.707589,0.764871,0.478947,1.029477,0.031360,-0.692334,0.021563
7,1.311696,-0.818618,-0.848003,1.094242,0.031675,-0.548413,1.406680,-0.082617,-0.041180,0.539934,-0.084694,-0.127199,-0.643116,-0.712277,0.002760
8,4.975225,-1.260772,-0.784167,0.349917,0.371137,0.442059,0.023706,-0.962922,-1.265593,0.885879,-0.455511,1.471954,-0.627624,-0.062308,0.025931
9,7.306996,2.425049,0.192555,0.799004,0.387296,-0.471431,-0.603250,-3.387402,0.012029,-0.245622,-0.054690,-0.383047,-0.408138,0.761225,0.357249


In [7]:
pd.DataFrame(x_data)

,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,aspect_ratio,budget,cast_total_facebook_likes,director_facebook_likes,duration,facenumber_in_poster,imdb_score,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year
3,1.608812,4.861555,10.197209,0.874036,0.631415,5.783711,5.771233,2.490684,-0.686152,1.888106,5.941166,4.507828,6.656297,6.220060,0.749048
5,-0.586410,-0.363660,-0.166965,0.874036,0.674870,-0.603069,-0.173248,1.040161,-0.195531,0.184910,0.326354,2.014444,1.218759,0.618968,0.749048
7,-0.573168,-0.382114,-0.280432,-0.974947,0.663134,-0.593143,-0.300284,-0.410363,-0.195531,1.260613,0.526883,1.034139,0.246484,1.115339,0.546496
9,1.442255,2.058328,4.201025,0.874036,0.631415,2.860502,-0.226548,1.992067,0.785711,0.991687,-0.235127,1.396425,1.869712,1.277489,0.445220
13,2.691432,0.656715,0.049820,0.874036,0.552119,2.235318,-0.148945,1.901409,0.295090,0.812403,-0.435656,0.955998,4.249154,2.480741,0.141392
14,2.691432,-0.044092,0.049820,0.874036,0.520400,2.069142,-0.148945,1.856080,-0.195531,0.095268,1.288893,1.929200,1.143969,0.436226,0.850325
18,2.691432,2.058328,0.049820,0.874036,0.631415,2.576133,-0.234833,1.221476,1.276333,0.274552,1.689951,1.914992,0.515175,1.571378,0.647772
19,0.193077,-0.320677,-0.080251,-0.974947,0.552119,0.048421,-0.252508,-0.138390,-0.195531,0.364194,0.968047,1.936303,0.119063,0.955166,0.749048
21,0.609470,1.824726,0.032754,0.874036,0.567978,1.017649,-0.176286,1.992067,-0.686152,0.543478,1.609740,2.987645,2.567756,2.058694,0.749048
27,1.109141,3.927146,4.662270,0.874036,0.631415,3.228598,-0.278467,1.720094,-0.686152,1.619180,2.251432,2.398041,2.005443,0.982302,1.154153


In [5]:
pd.DataFrame(pcaa.components_,columns=x_data.columns)

,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,aspect_ratio,budget,cast_total_facebook_likes,director_facebook_likes,duration,facenumber_in_poster,imdb_score,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,title_year
0,0.263667,0.294270,0.258920,0.159945,0.034300,0.324141,0.221291,0.203042,0.023484,0.235961,0.328778,0.352895,0.336059,0.360294,0.152372
1,0.376692,0.362921,0.331309,-0.002402,-0.033358,0.435773,-0.209145,-0.194509,0.221906,-0.293154,-0.155830,-0.169051,-0.274574,-0.263349,0.106054
2,0.084407,0.142986,0.153891,-0.432252,-0.037837,0.128440,0.284681,0.185930,-0.065445,0.151842,-0.259303,-0.300010,0.100024,0.126509,-0.646352
3,0.044325,0.046640,0.058986,-0.337193,0.480572,0.059225,0.045783,-0.435483,-0.633181,-0.139825,0.040486,0.112795,0.057056,0.039292,0.113368
4,-0.024296,-0.000252,-0.015101,0.108517,0.866391,-0.022648,-0.089467,0.310089,0.318598,0.086644,-0.064487,-0.044399,-0.075063,-0.074108,-0.076365
5,0.444489,-0.146391,-0.342174,0.336784,-0.028584,0.223289,-0.088236,0.347416,-0.524846,0.154476,-0.146998,-0.100001,-0.140638,-0.140759,-0.028408
6,0.271701,-0.122238,-0.304338,-0.552503,-0.043657,0.104511,-0.377960,-0.095449,0.228899,0.449779,0.216450,0.165664,-0.135770,0.004277,0.033450
7,0.368391,-0.307085,-0.386685,0.064711,0.085208,0.116811,0.517188,-0.344727,0.335752,-0.199587,-0.132425,-0.049066,0.128703,0.137308,0.033493
8,-0.106255,0.129814,0.035295,-0.174571,-0.017776,-0.047152,0.611782,0.129360,-0.025877,0.301313,0.026227,-0.018851,-0.466062,-0.306846,0.377779
9,0.087646,-0.120997,-0.092643,-0.366800,-0.036966,0.023434,0.069874,0.514523,-0.007818,-0.641640,0.352076,0.107552,0.015991,-0.120402,0.031323


In [ ]:
pcaa = decomposition.PCA(n_components=13)
x_9d = pcaa.fit_transform(x_data)
pd.DataFrame(x_9d)

In [ ]:
for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
                standard_scaler = preprocessing.StandardScaler().fit(dta_clean[colname])
                dta_clean[colname] = standard_scaler.transform(dta_clean[colname])

for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
        g = sns.lmplot(x=colname, y="gross", data=dta_clean, order=1)  


In [ ]:
for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
                standard_scaler = preprocessing.StandardScaler().fit(dta_clean[colname])
                dta_clean[colname] = standard_scaler.transform(dta_clean[colname])

for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
                poly = PolynomialFeatures(2).fit(dta_clean[colname])
                dta_clean[colname] = poly.transform(dta_clean[colname])              

for colname, colvalue in dta_clean.iteritems():
    if colname != 'gross' and colname != 'gross_class':
        g = sns.lmplot(x=colname, y="gross", data=dta_clean, order=2)  

In [2]:
poly = PolynomialFeatures(2)
poly_fit = poly.fit_transform(dta_clean) 

In [3]:
target_feature_names = ['x'.join(['{}^{}'.format(pair[0],pair[1]) for pair in tuple if pair[1]!=0]) for tuple in [zip(dta_clean.columns,p) for p in poly.powers_]]
pd.DataFrame(poly_fit, columns = target_feature_names)

,,actor_1_facebook_likes^1,actor_2_facebook_likes^1,actor_3_facebook_likes^1,aspect_ratio^1,budget^1,cast_total_facebook_likes^1,director_facebook_likes^1,duration^1,facenumber_in_poster^1,...,num_user_for_reviews^2,num_user_for_reviews^1xnum_voted_users^1,num_user_for_reviews^1xtitle_year^1,num_user_for_reviews^1xgross_class^1,num_voted_users^2,num_voted_users^1xtitle_year^1,num_voted_users^1xgross_class^1,title_year^2,title_year^1xgross_class^1,gross_class^2
0,1.0,27000.0,23000.0,23000.0,2.35,250000000.0,106759.0,22000.0,164.0,0.0,...,7295401.0,3.090854e+09,5434412.0,13505.0,1.309507e+12,2.302406e+09,5721685.0,4048144.0,10060.0,25.0
1,1.0,640.0,632.0,530.0,2.35,263700000.0,1873.0,475.0,132.0,1.0,...,544644.0,1.566066e+08,1484856.0,2952.0,4.503054e+10,4.269544e+08,848816.0,4048144.0,8048.0,16.0
2,1.0,799.0,553.0,284.0,1.85,260000000.0,2036.0,15.0,100.0,1.0,...,149769.0,1.140915e+08,777870.0,1935.0,8.691294e+10,5.925681e+08,1474050.0,4040100.0,10050.0,25.0
3,1.0,25000.0,11000.0,10000.0,2.35,250000000.0,58753.0,282.0,153.0,3.0,...,946729.0,3.131065e+08,1954757.0,4865.0,1.035520e+11,6.464862e+08,1608975.0,4036081.0,10045.0,25.0
4,1.0,40000.0,5000.0,1000.0,2.35,225000000.0,48486.0,563.0,151.0,2.0,...,3356224.0,9.563773e+08,3674992.0,9160.0,2.725258e+11,1.047212e+09,2610200.0,4024036.0,10030.0,25.0
5,1.0,40000.0,2000.0,1000.0,2.35,215000000.0,45757.0,563.0,150.0,1.0,...,505521.0,1.292541e+08,1431243.0,2844.0,3.304833e+10,3.659473e+08,727168.0,4052169.0,8052.0,16.0
6,1.0,40000.0,11000.0,1000.0,2.35,250000000.0,54083.0,252.0,136.0,4.0,...,234256.0,1.794207e+08,973324.0,2420.0,1.374215e+11,7.454857e+08,1853520.0,4044121.0,10055.0,25.0
7,1.0,10000.0,816.0,718.0,1.85,225000000.0,12572.0,188.0,106.0,1.0,...,116281.0,9.144051e+07,686092.0,1364.0,7.190657e+10,5.395258e+08,1072616.0,4048144.0,8048.0,16.0
8,1.0,15000.0,10000.0,963.0,2.35,230000000.0,28489.0,464.0,153.0,0.0,...,1500625.0,5.534587e+08,2464700.0,6125.0,2.041260e+11,9.090276e+08,2259015.0,4048144.0,10060.0,25.0
9,1.0,21000.0,19000.0,11000.0,2.35,250000000.0,64798.0,94.0,147.0,0.0,...,1044484.0,2.786687e+08,2060352.0,5110.0,7.434893e+10,5.497027e+08,1363350.0,4064256.0,10080.0,25.0


In [ ]:
poly

In [ ]:
dir(poly)

In [ ]:
poly.powers_

In [ ]:
poly.n_output_features_